In [42]:
import mmcv
import os
from PIL import Image

import math as m

import h5py
import numpy as np
#import matplotlib.pyplot as plt
import os
import torch
import cv2
from torchsummary import summary
from torch.autograd import Variable
#import torchvision
from torchvision import transforms
#from torch.utils.data import Dataset
import torch.nn as nn
import math
from tqdm import tqdm
#from focal_loss import FocalLoss
#from models.resnet_me import ResNet
use_gpu = torch.cuda.is_available()
device_ids=[0]        
#TEST
from models.SENet.se_resnet import se_resnet18,se_resnet34,se_resnet50

## 加载分类模型

In [45]:
model = se_resnet18(16)  

if use_gpu and len(device_ids)>1:#多gpu训练
    model = model.cuda(device_ids[0])
    model = nn.DataParallel(model, device_ids=device_ids)
if use_gpu and len(device_ids)==1:#单gpu训练
    model = model.cuda()
print(model)  
#print(summary(model,input_size = (3,64,64)))
#加载数据

# test_file = h5py.File('/home/zj/senetial/data/round2_test_a_20190121.h5', 'r')
# test_sen1 = test_file['sen1']
# test_sen2 = test_file['sen2']
# count=test_sen1.shape[0]
# print(count)

#加载模型
pretained_model = torch.load('/home/zj/senetial/save_models/SENet34/SENet34_96.pth')
model.load_state_dict(pretained_model)

# batchsize=128
# #保存结果
# f=open('/home/zj/senetial/data/SENet101_SNA_07_0.9805.csv','w')
# for i in tqdm(range(int(math.ceil(float(count)/batchsize)))):###不用管这里到底能不能整除，你进行向上取整就好
#     onehot = np.zeros((batchsize,17),dtype=int)
#     images=mean_std(AddFeatures(np.concatenate((test_sen1[i*batchsize:(i+1)*batchsize,:,:,:],test_sen2[i*batchsize:(i+1)*batchsize,:,:,:]),axis=3)))#列表：可以自动识别是否到了最后
#     inputs = Variable((torch.from_numpy(images)).float().permute(0, 3, 1, 2).cuda(device_ids[0]))
#     #print model
#     outputs = model(inputs)
#     scores, preds = torch.max(outputs.data, 1)#preds
#     for i in range(inputs.shape[0]):#注意这里不能用batchsize，只能用列表[:]后的shape
#         onehot[i,preds[i]]=1
#         f.writelines(','.join(map(str,onehot[i]))+'\n')
#         #f.writelines(','.join(map(str,onehot[i]))+','+str(scores[i])+'\n')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (sn1): SwitchNorm2d()
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (sn2): SwitchNorm2d()
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=4, bias=True)
          (1): ReLU(inplace)
          (2): Linear(in_features=4, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), strid

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [63]:
#10类的结果
dets = mmcv.load('/home/zj/mmdetection-swtx/work_dirs/cascade_rcnn_dconv_c3-c5_r50_fpn_1x/result_18.pkl')
new_dets = dets
#图片读取路径
f = open('/home/zj/mmdetection-swtx/data/VOCdevkit/VOC2007/ImageSets/Main/test (复件).txt','r')
lines = f.readlines()
f.close()
print(lines[0][:-1])
test_root = '/home/zj/mmdetection-swtx/data/VOCdevkit/VOC2007/JPEGImages'
print(len(new_dets))
new_dets = np.asarray(new_dets)
new_dets[0]

001752
2097


array([array([], shape=(0, 5), dtype=float32),
       array([[7.3562317e+02, 2.1623064e+02, 7.6431018e+02, 2.3472412e+02,
        9.9878067e-01],
       [7.0995581e+02, 2.3365616e+02, 7.6476361e+02, 2.8597391e+02,
        9.9888092e-01],
       [6.6730133e+02, 3.1940390e+02, 6.8176855e+02, 3.3179498e+02,
        1.6012287e-01]], dtype=float32),
       array([[5.75149414e+02, 3.48011108e+02, 6.18042725e+02, 3.83258911e+02,
        9.90883291e-01],
       [7.67795349e+02, 3.54260681e+02, 8.98865173e+02, 4.48436462e+02,
        9.98512208e-01],
       [6.00091675e+02, 3.54008057e+02, 6.92218018e+02, 4.32637299e+02,
        9.98820364e-01],
       [6.98875244e+02, 3.46846893e+02, 7.08746338e+02, 3.55391418e+02,
        2.19959572e-01],
       [6.45518677e+02, 3.45452515e+02, 6.62940125e+02, 3.56026581e+02,
        1.11476645e-01]], dtype=float32),
       array([], shape=(0, 5), dtype=float32),
       array([], shape=(0, 5), dtype=float32),
       array([], shape=(0, 5), dtype=float32),
   

In [5]:
# 首先需要解析结果，将0类的，再次输入给后续的分类，然后重新更改此处的分类，为其他的分类然后+1，其他的所有的都加16
for i in range(len(new_dets)):
    
    len_class0 = len(new_dets[i][0])
    if len_class0!=0:
        img = Image.open(os.path.join(test_root,lines[i][-1],'.jpg'))
        for x1,y1,x2,y2,s in new_dets[i][0]:
            cropped = img.crop((x1,y1,x2,y2))
            
            

FileNotFoundError: [Errno 2] No such file or directory: '/home/zj/mmdetection-swtx/data/VOCdevkit/VOC2007/JPEGImages/\n/.jpg'

In [6]:
x1,y1,x2,y2,score = new_dets[0][1][0]


In [22]:
img = Image.open(os.path.join(test_root,'001752.jpg'))
x1,y1,x2,y2,score = new_dets[0][1][0]
#cropped = img.crop((x1,y1,x2,y2))
cropped = img.crop((m.floor(x1),m.floor(y1),m.ceil(x2),m.ceil(y2)))
cropped.save('./1.jpg')
#np.asarray(cropped).shape
cropped

In [27]:
img = cv2.imread(os.path.join(test_root,'001752.jpg'))
x1,y1,x2,y2,score = new_dets[0][1][0]
cropped = img[m.floor(y1):m.ceil(y2),m.floor(x1):m.ceil(x2),:]
#cropped = img.crop((m.floor(x1),m.floor(y1),m.ceil(x2),m.ceil(y2)))
cv2.imwrite('./2.jpg',cv2.resize(cropped,(64,64)))
#np.asarray(cropped).shape

True

In [6]:
len(a)

2097

In [23]:
a[0][1][0]

array([735.62317   , 216.23064   , 764.3102    , 234.72412   ,
         0.99878067], dtype=float32)

735